In [ ]:
import os
import math
import pandas as pd
import seaborn as sn
import matplotlib.pyplot as plt
import numpy as np
from skimage.restoration import denoise_tv_bregman
#get inputs
spec_file = r"D:\git_MALDI\MALDI\230615PyMT-Fat-CMC-DHB-Pos-1-PyMT-Spectra-TIC.csv"
mass_file = r"D:\git_MALDI\MALDI\MassList-230615PyMT-Fat-CMC-DHB-Pos-1-PyMT.csv"
spot_file = r"D:\git_MALDI\MALDI\230615PyMT-Fat-CMC-DHB-Pos-1-RegionSpots.csv"

#UNCOMMENT THIS IF YOU WANT TO EXTRACT m/z FROM THE SPECTRA FILE
#os.system("grep m/z " + spec_file + " > mz")
    
#print("m/z is written in a file mz.")

f = open(r"D:\git_MALDI\MALDI\my_own\mz", "r")
mz = f.readline()
mz = mz.rstrip("\n")
mz = mz.split(";")
f.close() 
#os.system("rm mz")

for j in range(1, len(mz)):
    mz[j] = float(mz[j])

masses = pd.read_csv(mass_file)
masses.sort_values(by=['Masses from LIPID MAPS'], ascending = True, inplace = True)

out = pd.DataFrame(columns = ["Masses from LIPID MAPS", "Measured m/z", "m/z index", "Sphingolipid profile", "Common Name", "Adduct ion"])

k = 1
j = 0

## 10 ppm; If needed, this can be changed.
epsilon = 10E-6

while k < len(mz) and j < len(masses):
    m_exp = mz[k]
    m_theory = masses["Masses from LIPID MAPS"][j]
    
    if m_exp <= m_theory - epsilon * m_theory:
        k += 1
    elif abs(m_exp - m_theory) < epsilon * m_theory:
        temp = pd.DataFrame({"Masses from LIPID MAPS": [masses["Masses from LIPID MAPS"][j]],
                              "Measured m/z": [mz[k]],
                              "m/z index": [k],
                              "Sphingolipid profile": [masses["Sphingolipid profile"][j]],
                              "Common Name": [masses["Common Name"][j]],
                              "Adduct ion": [masses["Adduct ion"][j]]})
        
        if len(out) == 0:
            out = temp
        else:
            out = pd.concat([out, temp])
        k += 1
    else:
        j += 1

out.to_csv(r"D:\git_MALDI\MALDI\mass_temp_ind.csv", sep = ";", index = False)

print("A new mass list is written in mass_temp_ind.csv")

new_col =[0]
new_col.extend(out["m/z index"])

select_mz = pd.read_csv(spec_file, sep = ";", comment = "#",  usecols = new_col)

select_mz.to_csv(r"D:\git_MALDI\MALDI\spec_temp_ind.csv", sep = ";", index = False)

print("A new spec list is written in spec_temp_ind.csv")

from use_gen_map import gen_map

out_dir = r"D:\git_MALDI\MALDI\output_dir"
outputtype = "png"
spectra_filename = r"D:\git_MALDI\MALDI\spec_temp_ind.csv"
mass_filename = r"D:\git_MALDI\MALDI\mass_temp_ind.csv"
spots_filename = r"D:\git_MALDI\MALDI\230615PyMT-Fat-CMC-DHB-Pos-1-RegionSpots.csv"

#masses = pd.read_csv("masses_ind.csv")
masses = pd.read_csv(mass_filename, sep = ";")

maps_all = dict() 
added_map = np.zeros((1, 1))
v95_max = 0
for j in range(len(masses)):
    
    index = j + 1
    #print(j, "-".join(masses.iloc[j, 4].replace("/", ":").split(":")) + masses.iloc[j, 5][1:])
    #map, v95 = gen_map(index, "mz_data.csv", spots_filename)
    map, v95 = gen_map(index, mz_data_filename = spectra_filename, region_spots_filename = spots_filename, spectra_sep = ";", spot_sep = ";", space = 40)
    
    if added_map.size == 1:
        added_map = map
    
    if j > 0 and masses.iloc[j, 0] == masses.iloc[j - 1, 0]:
        added_map = added_map + map
        
    if (j == len(masses) - 1) or (masses.iloc[j, 0] != masses.iloc[j + 1, 0]):
        b = added_map.flatten()
        if sum(b > 0) < 1:
            v95_max = 0
        else:
            v95_max = np.percentile(b[b > -1], 95) 
        
        plt.figure(figsize=(6.4, 5.2))
        ax = sn.heatmap(added_map, vmax = v95_max, square = True, cmap = 'rainbow')
        plt.ylim(0, added_map.shape[0])
        title = "m/z: " + str(round(masses.iloc[j, 0], 4)) + '\n' + masses.iloc[j, 4] + masses.iloc[j, 5][1:]
        print(j, title)
        ax.set(xlabel = 'x (mm)', ylabel = 'y (mm)', title = title)
        plt.xticks(np.arange(0,201,50), np.arange(0,9,2))
        plt.yticks(np.arange(0,201,50), np.arange(0,9,2))
        plt.xticks(rotation=0)
        #plt.show()
        filename = out_dir + "/" + str(j) + "_" + "-".join(masses.iloc[j, 4].replace("/", ":").split(":")) + masses.iloc[j, 5][1:] + "." + outputtype
        plt.savefig(filename)
        print("Saving " + filename)
        # the following three commands clear too many open figures
        plt.cla()
        plt.clf()
        plt.close("all")

        ## denoising
        # record no data region
        no_data = []
        for l2 in range(added_map.shape[0]):
            for n2 in range(added_map.shape[1]):
                if math.isnan(added_map[l2, n2]): 
                    no_data.append((l2, n2))
                    added_map[l2, n2] = 0
        plt.figure(figsize=(6.4, 5.2))
        after_noise = denoise_tv_bregman(added_map, weight=0.03)
        
        # restore no data region
        for l2, n2 in no_data:
            after_noise[l2, n2] = np.nan
        b = after_noise.flatten()
        if sum(b > 0) < 1:
            v95_max = 0
        else:
            v95_max = np.percentile(b[b > -1], 95)
            
        if v95_max > 0:
            maps_all[masses.iloc[j, 4] + masses.iloc[j, 5]] = after_noise
        
        ax = sn.heatmap(after_noise, vmax = v95_max, square = True, cmap = 'rainbow')
        plt.ylim(0, added_map.shape[0])
        title = "m/z: " + str(round(masses.iloc[j, 0], 4)) + '\n' + masses.iloc[j, 4] + masses.iloc[j, 5][1:]
        #print(j, title)
        ax.set(xlabel = 'x (mm)', ylabel = 'y (mm)', title = title)
        plt.xticks(np.arange(0,201,50), np.arange(0,9,2))
        plt.yticks(np.arange(0,201,50), np.arange(0,9,2))
        plt.xticks(rotation=0)
        #plt.show()
        filename = out_dir + "/" + str(j) + "_" + "-".join(masses.iloc[j, 4].replace("/", ":").split(":")) + masses.iloc[j, 5][1:] + "_denoised." + outputtype
        plt.savefig(filename)
        print("Saving " + filename)
        # the following three commands clear too many open figures
        plt.cla()
        plt.clf()
        plt.close("all")
        
        ##
        added_map = np.zeros((1, 1))
        v95_max = 0

import pickle

with open(r'D:\git_MALDI\MALDI\my_own\saved_dictionary.pkl', 'wb') as f:
    pickle.dump(maps_all, f)

with open('saved_dictionary.pkl', 'rb') as f:
    loaded_dict = pickle.load(f)

print(loaded_dict)